In [ ]:
## Importing packages

import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import plot_model
from IPython.display import Image
from IPython.display import clear_output

data=pd.read_csv('{}'.format(df.csv))
X=data['features']
Y=data['Targets']
X_train,y_train,X_test,y_test=train_test_split(X,Y,test_size=0.2,random_state=123)

In [ ]:
## creating Y_train/Y_test targets for both classification and regression
y_train_reg=y_train['Reg_Target']
y_train_clf=y_train['Clf_Target']
y_test_reg=y_test['Reg_Target']
y_test_clf=y_test['Clf_Target']

In [ ]:
class PlotProgress(keras.callbacks.Callback):
    
    def __init__(self, entity='loss'):
        self.entity = entity
        
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('{}'.format(self.entity)))
        self.val_losses.append(logs.get('val_{}'.format(self.entity)))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="{}".format(self.entity))
        plt.plot(self.x, self.val_losses, label="val_{}".format(self.entity))
        plt.legend()
        plt.show();

In [ ]:
## creating 2 layer neural network with 64 neurons each and 2 output layers

input_layer = Input(shape=(X_train.shape[1],))
dense_layer_1 = Dense(64, activation='relu')(input_layer)
dense_layer_2 = Dense(64, activation='relu')(dense_layer_1)
regression = Dense(1, name='regression')(dense_layer_2)
classification=Dense(1, activation='sigmoid', name='classification')(dense_layer_2)
model = Model(inputs = input_layer,
     outputs = [regression,classification])
model.summary()

## Loss=weight1 * Loss1+weight2 * Loss2
model.compile(loss={'regression':'mean_absolute_error','classification':'binary_crossentropy'}, optimizer="adam"
              , loss_weights={'regression': 1, 'classification': 1},
              metrics={'regression':'mean_absolute_error','classification':'accuracy'})

In [ ]:
plot_progress = PlotProgress(entity='loss')

try:
    model.fit({'main_input': X_train},
              {'regression': y_train_reg, 'classification': y_train_clf},
              epochs=100, batch_size=128,
              verbose=1,
              callbacks=[plot_progress],
              validation_split=0.2,
             )
except KeyboardInterrupt:
    pass

In [ ]:
mae,proba=model.predict(X_test)